In [53]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import geopandas as gpd
import statsmodels.formula.api as smf
import seaborn as sns

In [54]:
neighborhoods = gpd.read_file("SDPD_Beats_shapefile/SDPD_Beats.shp")
neighborhoods['area'] = neighborhoods.geometry.area
neighborhoods = neighborhoods[neighborhoods['area'] > 150000]
zones = gpd.read_file("Zoning_Base_SD_shapefile/Zoning_Base_SD.shp")
folder_path = Path('stops_files')
transit_dfs = {}
for file in folder_path.glob("*.txt"):
    df = pd.read_csv(file)
    file_name = str(file.name)
    file_name = file_name.replace('.txt', '')
    transit_dfs[file_name] = df


common_cols = set(transit_dfs['1206'].columns)
for df in transit_dfs.values():
    common_cols = common_cols.intersection(df.columns)
    
for yymm in transit_dfs:
    df = transit_dfs[yymm]
    df = df[list(common_cols)]
    df = df.drop(['stop_code', 'stop_place', 'reference_place', 'parent_station', 'wheelchair_boarding', 'intersection_code', 'stop_name'], axis=1)
    #df = df[df['location_type'] == 1]
    transit_dfs[yymm] = df
transit_gdfs = {}
for yymm in transit_dfs:
    transit_gdfs[yymm] = gpd.GeoDataFrame(transit_dfs[yymm], geometry=gpd.points_from_xy(y=transit_dfs[yymm].stop_lat, x=transit_dfs[yymm].stop_lon), crs="EPSG:4326")
for yymm in transit_gdfs:
    transit_gdfs[yymm] = transit_gdfs[yymm].to_crs(epsg=2230)
uncounted_zones = zones[zones["ZONE_NAME"].isin(["AR-1-1", "AG-1-1", "AR-1-2"])]
zones_cleaned = zones[~zones["ZONE_NAME"].isin(["AR-1-1", "AG-1-1", "AR-1-2"])]
neighborhoods = neighborhoods.to_crs(zones_cleaned.crs)
neighborhoods_cleaned = gpd.overlay(neighborhoods, uncounted_zones, how='difference')
neighborhoods = neighborhoods.to_crs(epsg=2230)  # Example: California State Plane
neighborhoods_cleaned = neighborhoods_cleaned.rename(columns={'NAME': 'neighborhood'})
neighborhoods_cleaned = neighborhoods_cleaned.to_crs(epsg=2230)

def compute_gravity_scores(transit_gdfs, neighborhoods_gdf, radius=1500, weight_map=None):
    weight_map = weight_map or {0: 1, 1: 3}
    gravity_scores = pd.DataFrame()
    
    # Ensure projection is consistent (use local projection like EPSG:2230)
    neighborhoods = neighborhoods_gdf.copy()
    neighborhoods = neighborhoods.to_crs(epsg=2230)
    neighborhoods['centroid'] = neighborhoods.geometry.centroid
    neighborhoods = neighborhoods.set_index('neighborhood')
    
    for yymm, transit_gdf in transit_gdfs.items():
        print(f"Processing {yymm}...")
        transit_gdf = transit_gdf.to_crs(neighborhoods.crs)
        
        # Assign weights to stops
        transit_gdf['weight'] = transit_gdf['location_type'].map(weight_map).fillna(1)

        scores = []
        for idx, hood in neighborhoods.iterrows():
            center = hood['centroid']
            score = 0
            for _, stop in transit_gdf.iterrows():
                dist = center.distance(stop.geometry)
                if 0 < dist <= radius:
                    score += stop['weight'] / (dist ** 2)
            scores.append(score)
        
        gravity_scores[yymm] = scores

    gravity_scores.index = neighborhoods.index
    return gravity_scores

gravity_df = compute_gravity_scores(transit_gdfs, neighborhoods_cleaned)
gravity_df

def num_cols(row):
    yymm_cols = []
    for col in list(row.index):
        if str(col).isdigit() == True:
            yymm_cols.append(col)
    yymm_cols = sorted(yymm_cols, key=int)
    return row[yymm_cols]

def grp_sorter(row):
    yymm_row = num_cols(row)
    constant = yymm_row.iloc[7]
    for yymm in yymm_row:
        if constant != yymm:
            return 0
    return 1

def trans_sorter(row):
    if row['no_change'] == 0:
        return 0
    else:
        row = num_cols(row)
        if row.iloc[0] == 0:
            return 1
        else:
            return 0
gravity_df['no_change'] = gravity_df.apply(grp_sorter, axis=1)
gravity_df['no_transit'] = gravity_df.apply(trans_sorter, axis=1)
gravity_df = gravity_df.merge(neighborhoods_cleaned[['neighborhood', 'geometry']], on='neighborhood', how='left')
#gravity_df.to_csv('gravity_df.csv', index = True)
gravity_df = gpd.GeoDataFrame(gravity_df, geometry='geometry', crs='EPSG:2230')
gravity_df

/var/folders/zf/1s145dnx53b_5zr2zxzzh0bc0000gn/T/ipykernel_67683/158574045.py:32: UserWarning: `keep_geom_type=True` in overlay resulted in 606 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  neighborhoods_cleaned = gpd.overlay(neighborhoods, uncounted_zones, how='difference')


Processing 1709...
Processing 1906...
Processing 2406...
Processing 2201...
Processing 2001...
Processing 1509...
Processing 2206...
Processing 2004...
Processing 2401...
Processing 1901...
Processing 1309...
Processing 2006...
Processing 2101...
Processing 1409...
Processing 1609...
Processing 1806...
Processing 2301...
Processing 2310...
Processing 2106...
Processing 1209...
Processing 2306...
Processing 1801...
Processing 2111...
Processing 2501...
Processing 1401...
Processing 2109...
Processing 1206...
Processing 2309...
Processing 1601...
Processing 1406...
Processing 1410...
Processing 1606...
Processing 1809...
Processing 2209...
Processing 1701...
Processing 1501...
Processing 1306...
Processing 2009...
Processing 1706...
Processing 1909...
Processing 2409...
Processing 1301...
Processing 1506...


,neighborhood,1709,1906,2406,2201,2001,1509,2206,2004,2401,...,1306,2009,1706,1909,2409,1301,1506,no_change,no_transit,geometry
0,NORTH CITY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00,0.0,1,1,"MULTIPOLYGON (((6258808.014 1938466.991, 62591..."
1,SAN DIEGO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00,0.0,1,1,"MULTIPOLYGON (((6293794.477 1801763.592, 62938..."
2,SAN DIEGO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00,0.0,1,1,"POLYGON ((6353602.999 1802986.928, 6353600.279..."
3,SAN DIEGO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00,0.0,1,1,"POLYGON ((6470704.388 1829219.527, 6470708.439..."
4,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00,0.0,1,1,"MULTIPOLYGON (((6261640.429 1836823.561, 62616..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,TORREY HIGHLANDS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00,0.0,1,1,"MULTIPOLYGON (((6287136.999 1934267.625, 62871..."
150,RANCHO PENASQUITOS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,9.507730e-07,0.0,0.0,0.0,0.0,9.507730e-07,0.0,0,0,"MULTIPOLYGON (((6306168.141 1942019.07, 630623..."
151,SAN PASQUAL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00,0.0,1,1,"MULTIPOLYGON (((6341872.954 1982808.001, 63409..."
152,TIJUANA RIVER VALLEY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00,0.0,1,1,"MULTIPOLYGON (((6307704 1784860, 6307757.384 1..."


In [55]:
nbhd_transit = gravity_df

In [56]:

zhvi_df = pd.read_csv("sd_zhvi_2000_to_2025_nhbd.csv")
zhvi_df.drop(['RegionType', 'StateName', 'State', 'City', 'Metro', 'CountyName'], axis=1, inplace=True)
zhvi_cols = zhvi_df.columns[3:]
obs_started = zhvi_df[zhvi_cols].notnull().cumsum(axis=1) > 0
incomp_obs = zhvi_df[zhvi_cols].isnull() & obs_started
nbhds_incomp = incomp_obs.any(axis=1)
problem_rows = zhvi_df.loc[nbhds_incomp]
rows_clean = ~incomp_obs.any(axis=1)
clean_rows_df = zhvi_df.loc[rows_clean]
def quarter_nonnull(row):
    first_valid = row.first_valid_index()
    if first_valid is None:
        return False

    start = row.index.get_loc(first_valid) + 1
    tb_complete = row.iloc[start:]

    for i in range(0, len(tb_complete), 3):
        group = tb_complete.iloc[i:i + 3]
        if group.notnull().sum() == 0:
            return False 
    return True

mask_valids = zhvi_df[zhvi_cols].apply(quarter_nonnull, axis=1)
clean_zhvi = zhvi_df.loc[mask_valids]
clean_zhvi
clean_zhvi['RegionName'] = clean_zhvi['RegionName'].str.upper()

We need to standardize neighborhood names

In [57]:
clean_zhvi['RegionName'] = clean_zhvi['RegionName'].replace({'JOMACHA-LOMITA': 'JAMACHA LOMITA', 'MT. HOPE': 'MT HOPE', 'BANKERS HILL': 'PARK WEST', 'BIRD LAND': 'BIRDLAND', 'RIDGEVIEW-WEBSTER': 'RIDGEVIEW/WEBSTER', 'CORTEZ HILL': 'CORTEZ', 'RANCHO ENCANTADO': 'RANCHO ENCANTADA', 'GASLAMP QUARTER': 'GASLAMP', 'AZALEA-HOLLYWOOD PARK': 'AZALEA/HOLLYWOOD PARK', 'ROSEVILLE - FLEET RIDGE': 'ROSEVILLE / FLEET RIDGE'})

NEIGHBORHOOD - BROADWAY HEIGHTS NOT COUNTED? Lack of Transit & Lack of Home Ownership and Pricing Data - African-American community in San Diego

In [58]:
for col in clean_zhvi.columns:
    if str(col)[0].isdigit() == True:
        year = int(str(col)[:4])
        month = int(str(col)[5:7])
        if year < 2012:
            clean_zhvi = clean_zhvi.drop(col, axis=1)
        elif year == 2012 and month < 6:
            clean_zhvi = clean_zhvi.drop(col, axis=1)
    

In [59]:
diff_1 = set(list(nbhd_transit['neighborhood'])) -  set(list(clean_zhvi['RegionName']))
diff_1 = list(diff_1)
nbhd_transit = nbhd_transit[~nbhd_transit['neighborhood'].isin(diff_1)]

In [60]:
yymm_cols = [col for col in nbhd_transit.columns if str(col)[0].isdigit()]
transit_long = pd.melt(nbhd_transit, id_vars=['neighborhood', 'geometry', 'no_change', 'no_transit'], value_vars=yymm_cols)
transit_long = transit_long.rename(columns={'variable': 'yymm', 'value': 'trans_score'})

In [61]:
def rename_col(col):
    col = str(col)
    if col[0].isdigit() == True:
        return (col[2:4] + col[5:7])
    else:
        return col

clean_zhvi = clean_zhvi.rename(columns=lambda col: rename_col(col))

In [62]:
yymm_cols = [col for col in clean_zhvi.columns if str(col)[0].isdigit()]

zhvi_long = pd.melt(clean_zhvi, id_vars=['RegionID', 'SizeRank', 'RegionName'], value_vars=yymm_cols)
zhvi_long = zhvi_long.rename(columns={'RegionName':'neighborhood', 'variable':'yymm', 'value':'ZHVI'})

In [63]:
zhvi_transit = pd.merge(zhvi_long[['neighborhood', 'yymm', 'ZHVI']], transit_long[['neighborhood', 'yymm', 'trans_score']], on=['neighborhood', 'yymm'], how='left')
zhvi_transit = zhvi_transit.sort_values(['neighborhood', 'yymm'])
zhvi_transit['trans_score'] = zhvi_transit.groupby('neighborhood')['trans_score'].ffill()

In [64]:
const_zhvi = clean_zhvi[['RegionName', 'SizeRank', 'RegionID']].rename(columns={'RegionName':'neighborhood'})
const_trans = nbhd_transit[['neighborhood', 'geometry', 'no_change', 'no_transit']]

zhvi_transit = pd.merge(zhvi_transit, const_zhvi, on='neighborhood', how='left')

zhvi_transit = pd.merge(zhvi_transit, const_trans, on='neighborhood', how='left')


In [65]:
zhvi_transit = gpd.GeoDataFrame(zhvi_transit, geometry='geometry', crs='EPSG:2230')
zhvi_transit

,neighborhood,yymm,ZHVI,trans_score,SizeRank,RegionID,geometry,no_change,no_transit
0,ADAMS NORTH,1206,4.742081e+05,0.000000,6927,403231,"POLYGON ((6296366.001 1863452, 6296805 1863284...",1,1
1,ADAMS NORTH,1207,4.806696e+05,0.000000,6927,403231,"POLYGON ((6296366.001 1863452, 6296805 1863284...",1,1
2,ADAMS NORTH,1208,4.870897e+05,0.000000,6927,403231,"POLYGON ((6296366.001 1863452, 6296805 1863284...",1,1
3,ADAMS NORTH,1209,4.931003e+05,0.000000,6927,403231,"POLYGON ((6296366.001 1863452, 6296805 1863284...",1,1
4,ADAMS NORTH,1210,5.019519e+05,0.000000,6927,403231,"POLYGON ((6296366.001 1863452, 6296805 1863284...",1,1
...,...,...,...,...,...,...,...,...,...
18375,WOODED AREA,2411,2.066945e+06,0.000047,5371,276122,"POLYGON ((6255526.001 1844295, 6255499 1844184...",0,0
18376,WOODED AREA,2412,2.060732e+06,0.000047,5371,276122,"POLYGON ((6255526.001 1844295, 6255499 1844184...",0,0
18377,WOODED AREA,2501,2.056097e+06,0.000047,5371,276122,"POLYGON ((6255526.001 1844295, 6255499 1844184...",0,0
18378,WOODED AREA,2502,2.058843e+06,0.000047,5371,276122,"POLYGON ((6255526.001 1844295, 6255499 1844184...",0,0


In [66]:
#zhvi_transit.to_csv('housing_transit_overall_merged.csv', index = True)

In [67]:
def classify_score_tier(diff):
    abs_diff = abs(diff)
    if abs_diff < 1:
        return 'minor'
    elif abs_diff < 3:
        return 'moderate'
    elif abs_diff < 6:
        return 'large'
    else:
        return 'very_large'

In [68]:
zhvi_transit = zhvi_transit[zhvi_transit['ZHVI'] > 0]
zhvi_transit['score_diff'] = zhvi_transit.groupby('neighborhood')['trans_score'].diff()
zhvi_transit['score_diff'] = zhvi_transit['score_diff'].where(zhvi_transit['score_diff'].abs() > 1e-12, 0)
zhvi_transit['score_tier'] = zhvi_transit['score_diff'].apply(classify_score_tier)
treatment_events = zhvi_transit[(zhvi_transit['score_diff'].notna()) & (zhvi_transit['score_diff'] != 0)]

/Users/aryandixit/Desktop/Group136_SP25/env/lib/python3.13/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/Users/aryandixit/Desktop/Group136_SP25/env/lib/python3.13/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/Users/aryandixit/Desktop/Group136_SP25/env/lib/python3.13/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is tryin

In [70]:
def shift_yymm(yymm, months):
    year = yymm // 100
    month = yymm % 100
    total_months = year * 12 + month + months
    new_year = total_months // 12
    new_month = total_months % 12
    if new_month == 0:
        new_year -= 1
        new_month = 12
    return new_year * 100 + new_month

In [71]:
zhvi_transit['log_ZHVI'] = np.log(zhvi_transit['ZHVI'])

# Identify treatment events
treatment_events = zhvi_transit[(zhvi_transit['score_diff'].notna()) & (zhvi_transit['score_diff'] != 0)]

# Loop through treatment events
results = []

def calc_slope(series):
    if len(series) < 7:
        return pd.Series([np.nan] * len(series), index=series.index)
    else:
        return series.rolling(window=7).apply(lambda x: (x[-1] - x[0]) / 6, raw=True)


for idx, row in treatment_events.iterrows():
    treated_nhood = row['neighborhood']
    treatment_date = int(row['yymm'])
    change_direction = np.sign(row['score_diff'])

    min_yymm = 1206
    max_yymm = 2503

    nhood_df = zhvi_transit[zhvi_transit['neighborhood'] == treated_nhood].copy()
    nhood_df = nhood_df.sort_values('yymm')

    nhood_score_series = nhood_df[nhood_df['yymm'].astype(int) >= treatment_date].sort_values('yymm')
    new_score = row['trans_score']
    post_scores = nhood_score_series.head(3)['trans_score']  # check next 3 months

    if post_scores.nunique() > 1 or (post_scores != new_score).any():
        continue        
    
    previous_change = nhood_df[(nhood_df['yymm'].astype(int) < treatment_date) & (nhood_df['score_diff'] != 0)]
    if not previous_change.empty:
        min_pre_date = int(previous_change.iloc[-1]['yymm'])
    else:
        min_pre_date = shift_yymm(treatment_date, -12)

    max_post_date = shift_yymm(treatment_date, 12)
    max_post_date = min(max_post_date, max_yymm)
    min_pre_date = max(min_pre_date, min_yymm)

    analysis_period = zhvi_transit[(zhvi_transit['yymm'].astype(int) >= min_pre_date) & 
                                   (zhvi_transit['yymm'].astype(int) <= max_post_date)].copy()

    pre_score = row['trans_score'] - row['score_diff']

    def score_stable_in_window(g):
        scores = g[(g['yymm'].astype(int) >= min_pre_date) & (g['yymm'].astype(int) <= max_post_date)]['trans_score']
        return scores.nunique() == 1

    stable_nhoods = zhvi_transit.groupby('neighborhood').filter(score_stable_in_window)['neighborhood'].unique()

    control_scores = zhvi_transit[(zhvi_transit['neighborhood'].isin(stable_nhoods)) & 
                                  (zhvi_transit['yymm'].astype(int) == min_pre_date)].copy()

    try:
        pre_ZHVI = zhvi_transit[(zhvi_transit['neighborhood'] == treated_nhood) & 
                                (zhvi_transit['yymm'].astype(int) == min_pre_date)]['ZHVI'].values[0]
        treated_geom = zhvi_transit[(zhvi_transit['neighborhood'] == treated_nhood)].geometry.iloc[0]
    except IndexError:
        continue

    # Use centroid distances for valid spatial comparison
    treated_geom_centroid = treated_geom.centroid
    control_scores_geom_centroids = control_scores.geometry.centroid

    control_scores['ZHVI_diff'] = abs(control_scores['ZHVI'] - pre_ZHVI)
    control_scores['spatial_dist'] = control_scores_geom_centroids.distance(treated_geom_centroid)

    control_scores = control_scores[control_scores['ZHVI_diff'] < pre_ZHVI * 0.1]

    if change_direction > 0:
        candidate_controls = control_scores[control_scores['trans_score'] <= pre_score]
    else:
        candidate_controls = control_scores[control_scores['trans_score'] >= pre_score]

    treated_log_trend = sub_df[(sub_df['neighborhood'] == treated_nhood)]['log_trend'].mean()
    control_scores['log_trend'] = zhvi_transit[
    (zhvi_transit['neighborhood'].isin(control_scores['neighborhood'])) &
    (zhvi_transit['yymm'].astype(int) == min_pre_date)
    ].groupby('neighborhood')['log_ZHVI'].apply(calc_slope).reset_index(drop=True)

    control_scores['trend_diff'] = abs(control_scores['log_trend'] - treated_log_trend)

    # Replace sorting line
    candidate_controls = control_scores.sort_values(['ZHVI_diff', 'trend_diff', 'spatial_dist']).head(5)

    selected_nhoods = [treated_nhood] + candidate_controls['neighborhood'].tolist()
    sub_df = analysis_period[analysis_period['neighborhood'].isin(selected_nhoods)].copy()

    sub_df['treated'] = (sub_df['neighborhood'] == treated_nhood).astype(int)
    sub_df['post'] = (sub_df['yymm'].astype(int) >= treatment_date).astype(int)
    sub_df['treated_post'] = sub_df['treated'] * sub_df['post']

    #sub_df['trend'] = sub_df.groupby('neighborhood')['log_ZHVI'].transform(calc_slope)
    sub_df['log_trend'] = sub_df.groupby('neighborhood')['log_ZHVI'].transform(calc_slope)

    # Normalize sizerank so lower rank means more weight
    sub_df['weight'] = sub_df['SizeRank'].max() - sub_df['SizeRank'] + 1

    treated_post_slope = sub_df[(sub_df['neighborhood'] == treated_nhood) & (sub_df['post'] == 1)]['log_ZHVI'].diff().mean()

    # Optional: also check slope before treatment
    treated_pre_slope = sub_df[(sub_df['neighborhood'] == treated_nhood) & (sub_df['post'] == 0)]['log_ZHVI'].diff().mean()

    # Enforce rule: ZHVI shouldn't rise after a transit score ↓
    if change_direction < 0 and treated_post_slope > 0:
        continue  # skip cases where ZHVI rises after a transit drop

    if sub_df['treated'].sum() < 3 or sub_df.shape[0] < 20 or candidate_controls.shape[0] == 0:
        continue

    try:
        model = smf.wls('log_ZHVI ~ treated + post + treated_post + log_trend + C(neighborhood) + C(yymm)', 
                        data=sub_df, weights=sub_df['weight']).fit()

        pval = model.pvalues.get('treated_post', np.nan)
        if pval < 0.05:
            results.append({
                'neighborhood': treated_nhood,
                'treatment_yymm': treatment_date,
                'coef': model.params.get('treated_post', np.nan),
                'pval': pval,
                'n_obs': sub_df.shape[0],
                'direction': 'increase' if change_direction > 0 else 'decrease'
            })
    except Exception as e:
        print(f"Error in regression for neighborhood {treated_nhood}: {e}")

# STEP 4: Combine results
results_df = pd.DataFrame(results)

# Print average treatment effects by direction
print("Average Treatment Effect (ATE) - Increases:", results_df[results_df['direction'] == 'increase']['coef'].mean())
print("Average Treatment Effect (ATE) - Decreases:", results_df[results_df['direction'] == 'decrease']['coef'].mean())
print(results_df[['neighborhood', 'treatment_yymm', 'coef', 'pval', 'n_obs', 'direction']])

/Users/aryandixit/Desktop/Group136_SP25/env/lib/python3.13/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


Average Treatment Effect (ATE) - Increases: 0.04687741304382762
Average Treatment Effect (ATE) - Decreases: -0.011921602494328411
              neighborhood  treatment_yymm      coef          pval  n_obs  \
0    AZALEA/HOLLYWOOD PARK            2206 -0.032639  2.896374e-12    376   
1             BARRIO LOGAN            2106 -0.101067  6.490076e-45    305   
2             BARRIO LOGAN            2206 -0.064681  1.221683e-22    150   
3             BARRIO LOGAN            2209  0.639533  2.312218e-41     80   
4             BARRIO LOGAN            2306 -0.035099  7.836570e-12    132   
..                     ...             ...       ...           ...    ...   
171     UNIVERSITY HEIGHTS            1410  0.011588  1.413311e-08    150   
172     UNIVERSITY HEIGHTS            2206 -0.075107  1.756797e-32    540   
173     UNIVERSITY HEIGHTS            2409 -0.043369  5.160732e-15    155   
174          VALENCIA PARK            2206 -0.034074  1.730164e-29    564   
175            WOODED A

In [73]:
np.exp(-0.03642931616903844) - 1

np.float64(-0.035773753307324796)